## 욕설/한국어 나무위키 크롤링

### selenium

In [ ]:
!pip install webdriver_manager


In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
from bs4 import BeautifulSoup
import re

In [ ]:
# DriverManager를 이용해 WebDriver생성
def load_page(url):
    driver = ChromeDriverManager().install() # 다운받은 경로 반환
    service = Service(excutable_path = driver) # driver 경로 넣어서 생성
    browser = webdriver.Chrome(service = service)
    browser.get(url)
    browser.fullscreen_window() # 창 크기 최대

    return browser

# 나무위키에서 비속어 데이터 거져오기
def get_content(browser):
    bad_words = []
    div_child = browser.find_elements(By.CSS_SELECTOR, '#app > main > div.OCdtrch3 > div.\+h5zJAxv > article > div.fak4patN > div > div:nth-child(2) > div > div > div')

    for child in div_child[2:17]:
        words_list = child.find_elements(By.CSS_SELECTOR, 'ul')

        for word in words_list:
            # 필요없는 단어 필터링
            bad_word = filtering(word.text)
            if not bad_word:
                print(bad_word)
                continue

            # 중복 데이터 제거
            if not bad_word in bad_words:
                bad_words.append(bad_word)


    # 중복 데이터 제거

            # words_list = word.find_elements(By.CSS_SELECTOR, 'li > div > a')
            # for w in words_list:
            #     if not w.text[0].startswith('['):
            #         bad_word.append(w.text)


    return bad_words


def filtering(word):
    # 한국어만 추출
    word = re.compile('[가-힣]+').findall(word)

    # 텍스트 형식 : 비속어 + 비속어 설명  ex) 간나: 간나새끼, 간새(간나새끼의 준말), 개간나, 쌍간나, 종간나, 좆간나 등
    # 비속어 설명 제외 비속어만 수집
    bad_word = word[0]

    # 직접 필터링 : 관련없는 데이터 제외
    filtering_words = [ '네', '개', '눈', '니', '닭', '너']
    if bad_word in filtering_words:
        return ""

    return bad_word

# 텍스트 파일로 저장
def save_txt(file_name, data):
    try:
        with open(file_name, 'w+') as file:
            for w in data:
                file.write(w+'\n')
    except:
        print('저장 실패')

    print('데이터 저장 ')

def concat_data(data_list : list):
    concat = []
    for data in data_list:
        concat += data

    return list(set(concat))


In [ ]:
# 페이지 불러오기
url = 'https://namu.wiki/w/%EC%9A%95%EC%84%A4/%ED%95%9C%EA%B5%AD%EC%96%B4'
browser = load_page(url)
# 비속어 가져오기
bad_words = get_content(browser)


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=121.0.6167.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000106d4d538 chromedriver + 4687160
1   chromedriver                        0x0000000106d44d83 chromedriver + 4652419
2   chromedriver                        0x0000000106935fbd chromedriver + 397245
3   chromedriver                        0x00000001069457e3 chromedriver + 460771
4   chromedriver                        0x000000010693c24a chromedriver + 422474
5   chromedriver                        0x000000010693a44e chromedriver + 414798
6   chromedriver                        0x000000010693d77f chromedriver + 427903
7   chromedriver                        0x000000010693d82c chromedriver + 428076
8   chromedriver                        0x000000010697

#### BeautifulSoup

In [ ]:
# 1번째 시도
url = "https://namu.wiki/w/%EC%9A%95%EC%84%A4/%ED%95%9C%EA%B5%AD%EC%96%B4"

resp = requests.get(url=url)

soup = BeautifulSoup(resp.text)

comp = re.compile('[ㄱ-ㅎㅏ-ㅣ가-힣]+')

rem_set = {'나무위키','검색','이동','한국어','욕설'}

test_li = list()

for a in soup('a'):
    # Title 있는거
    if 'title' in a.attrs and len(a['title'].split()) == 1 and bool(comp.fullmatch(a['title'])) and a['title'] not in rem_set:
        test_li.append(a['title'])
    # if 'title' not in a.attrs:
    #     print(a)

In [ ]:
#1. : 으로 단어 : 뜻
#2. 단어만
#3. / 로 구분
#4. ()안에 있는것.
#5. ,로 구분
#6. 특정 단어  (네 다음 XXㄴㄷㅆ),(니 XX(패드립)니애미),('닭/새/돌/빡+~대가리')

In [ ]:
#3번째 시도
test_li2 = list()
def refine_1(word):
    return word.split(':')[0].strip()

def refine_2(word):
    if '/' in word:
        for w in word.split('/'):
            test_li2.append(w.strip())
        return True
    else:
        return False

def refine_3(word):
    if ',' in word:
        for w in word.split(','):
            test_li2.append(w.strip())
        return True
    else:
        return False

def refine_4(word):
    if '(' in word:
        test_li2.append(word.split('(')[0])
        test_li2.append(word.split('(')[1][:-1])
        return True
    else:
        return False

eng_pat = re.compile('[a-z]+')

for ul in soup('ul'):
    # 1. :로 단어가 정의된 것 추출
    hate_word = refine_1(ul.text)
    if bool(eng_pat.search(hate_word)) or not hate_word:
        continue
    if refine_2(hate_word):
        continue
    if refine_3(hate_word):
        continue
    if refine_4(hate_word):
        continue
    test_li2.append(hate_word)

### 데이터 비교
- beautifulsoup 과 selenium 으로 수집한 데이터 비교

In [ ]:
# bs4
bs4_data = test_li2
selenium_data = bad_words
print(f'bs4_data : {len(bs4_data)}')
print(f'selenium_data : {len(selenium_data)}')


bs4_data : 295
selenium_data : 224


In [ ]:
differ = list(set(bs4_data) - set(selenium_data))
differ

['애비 뒤진 년 (애미',
 '짜져 있어',
 '허접쓰레기 또는 허섭스레기',
 '거렁뱅이',
 '나가 뒤져',
 '변태새끼',
 '재수가 없다',
 '깝쳐',
 '니애비느금마',
 '찐찐찌버거',
 '말하는 가축',
 '病身',
 '제길',
 '혹은 딸딸이대',
 '광년이',
 '엠창(니미씨발)',
 '○레기',
 '버러지',
 '닭',
 '호로(자식)',
 '박쥐같은 새끼',
 '또는 왕쇠똥구리',
 '시발)',
 '쌍노무새끼',
 '~퀴',
 '육갑하다',
 '니 XX(패드립)니애미',
 '개새',
 '급식',
 '땡추',
 '창남',
 '후레자식',
 '싹(퉁)바가지',
 '개씹',
 '애비 터진 새끼)',
 '년',
 '새',
 '박쥐짓',
 '괴뢰군',
 '주둥이.주둥아리',
 '창놈',
 'ㄴㄷㅆ',
 '~충',
 '부랄',
 '쪽바리',
 '미친년',
 '개좆',
 '사이버 렉카',
 '씨방짭새',
 '미친개미친놈',
 '엄창엠창인생',
 '나가 죽어',
 '째져 있어',
 '니미',
 '옘병',
 '너 같은 새끼는 뻑큐가 최고야',
 '환향녀',
 '싸가지(가 없다)',
 '씨방놈',
 '개',
 '아치',
 '삼성빠돌이',
 '나이 든 미혼녀에 대한 욕설.',
 '쌍년',
 '한남, 한남충',
 '느개비',
 '부모 홀수인 새끼',
 '딸딸이 부대',
 '경을 칠 놈',
 '흑우',
 '너검엄빠',
 '추녀',
 '네 다음 XXㄴㄷㅆ',
 '틀',
 '빡+~대가리',
 '찌랭이',
 '장궤',
 '뒤지다',
 '상폐녀 2010년대 초 주식 갤러리에서 유래한',
 '한국어 욕설',
 '씨발 (씨팔',
 '짱꼴라',
 '돌',
 '씨방년',
 '한녀, 한녀충',
 '다신 이세상에 내려올 생각도 하지말어',
 '눈 깔아',
 "경상권에서는 '똘게이'라 한다"]

### 비속어 데이터 추가

In [ ]:
# 추가 데이터
# w = '흑우'
# w in selenium_data
add_words = [ '짱꼴라', '옘병', '니미', '흑우', '쪽바리', '괴뢰군', '씨방년', '거렁뱅이', '후레자식', '쌍년', '미친년', '광년이', '창놈', '찌랭이', '창남']

bad_words.extend(add_words)
namu_bad = bad_words
print(f'총 비속어 수 : {len(namu_bad)}')

총 비속어 수 : 239
데이터 저장 


### 위키백과 한국어 속어

In [ ]:
def get_data(url):
    resp = requests.get(url=url)
    soup = BeautifulSoup(resp.text)
    bad_words = list()
    word_tag = soup.select('#mw-pages > div > div > div:nth-child(1) > ul > li')
    for word in word_tag:
        bad_words.append(word.text)

    return bad_words


In [ ]:
url = 'https://ko.wiktionary.org/wiki/%EB%B6%84%EB%A5%98:%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%86%8D%EC%96%B4'
wiki_bad = get_data(url)
wiki_bad

['가오',
 '가오마담',
 '갈보',
 '갈토',
 '개년',
 '개놈',
 '개돼지',
 '개새끼',
 '개자식',
 '개지랄',
 '개판',
 '거짓부렁',
 '거짓부렁이',
 '거짓부리',
 '거짓불',
 '검둥이',
 '겁대가리',
 '겐세이',
 '계집년',
 '고딩',
 '고인물',
 '공갈',
 '공갈치다',
 '관종',
 '귀때기',
 '귀머거리',
 '금사빠',
 '급식',
 '깡통',
 '깽깽이',
 '껌둥이',
 '꼬붕',
 '꼭대기',
 '꼰대',
 '꼴값',
 '꼴값하다',
 '꼴다',
 '꼴리다',
 '꽃뱀',
 '꽝',
 '꿀잼']

나무위키와 위키백과 데이터 통합
- 데이터 수 변화 : 239 -> 271

In [ ]:
concat_word = concat_data([namu_bad, wiki_bad])
print(len(concat_word))

271


In [ ]:
save_txt('bad_word.txt', concat_word)

데이터 저장 


----

### 네이트판 댓글 크롤링

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
from bs4 import BeautifulSoup
import re

In [ ]:
# DriverManager를 이용해 WebDriver생성
def load_page(url):
    driver = ChromeDriverManager().install() # 다운받은 경로 반환
    service = Service(excutable_path = driver) # driver 경로 넣어서 생성
    browser = webdriver.Chrome(service = service)
    browser.get(url)
    browser.fullscreen_window() # 창 크기 최대

    return browser

In [ ]:
import time
from selenium.common.exceptions import StaleElementReferenceException


def nate_comment(browser):
    comment_list = []


    for rank_page in range(1,3):
        if rank_page == 1:
            last_button = browser.find_element(By.ID, 'last')
            last_button.click()
            time.sleep(5)

        time.sleep(3)
        # 각 페이지별 정보 가져오기
        posts = browser.find_elements(By.CSS_SELECTOR, 'div.tsCnt > div.cntList > ul > li')
        post_links = []

        # 50개 게시글 정보
        for post in posts:
            href_tag = post.find_element(By.CSS_SELECTOR, 'dl > dt > h2 > a')
            link = href_tag.get_attribute("href")
            post_links.append(link)

        # 상세 게시글 이동
        for page_link in post_links:
            comment_list.extend(get_comment(browser, page_link))

        url = 'https://pann.nate.com/Talk/Ranking?rankingType=enter'
        browser.get(url)

    return comment_list


def get_comment(browser, link):
    comment_list = list()
    browser.get(link)
    time.sleep(5)

    now_page = browser.find_element(By.CSS_SELECTOR, '#commentDiv > div.paginate-reple > strong')
    if now_page == 1:
        comments = WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.usertxt>span')))
        # 댓글 수집
        for c in comments:
            comment_list.append(c.text)
    else:
        # 다음 페이지
        # page_nums = WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.paginate-reple>a')))
        # if len(page_nums) == 1:
        #     page_nums = [page_nums]

        # for page in page_nums[:2]: # 에러발생 2페이까지만 수집
        #     try:
        #         #browser.execute_script("arguments[0].click();", page)
        #         page.click()
        #     except StaleElementReferenceException:
        #         print("StaleElementReferenceException 발생. 다시 찾습니다.")
        #         page = WebDriverWait(browser, 10).until(
        #             EC.element_to_be_clickable((By.CSS_SELECTOR, '.paginate-reple>a'))
        #         )
        #         browser.execute_script("arguments[0].click();", page)

        #     time.sleep(5)
            try:
                comments = WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.usertxt>span')))
                # 댓글 수집
                for c in comments:
                    comment_list.append(c.text)
                    print(c.text)
            except Exception :
                print(f'{Exception.args}')



    return comment_list


url = 'https://pann.nate.com/Talk/Ranking?rankingType=enter'
# 사이트맵 로드
browser = load_page(url)
comment_list = nate_comment(browser)


당연히 승한아님? 전에 여자를 만났든 뭘했든 지금 나한테 씨~발 거리진 않자녀
닥승한 팬들한텐 잘함
닥승한 얜 솔직히 지금 활중한것도 이해안감
굳이 이런 애들을 골라야함? 웅딱정이랑 모텔남 ㅠㅠ 개싫네
빠혐하는 아이돌이 실존해도되냐? 활동중도아닌 데뷔전 일반인시절에 한 게 왜 지금까지 쌍욕먹는 건지도 ㅉㅉ
승한 ㅇㅇ 이미지는 별로지만 빠혐은 안했으니
승한
승한 얜 인정하고 사과라도 했지
고능한 승한팬이노
웅덩이 빼고 승한이라함 ㅇㅇ과거 ㅁㅌ병크 라방 병크는 과거잖니 아이돌로 데뷔하고 영통 팬싸에서 그러고 사필귀정?ㅋㅋ
승한은 그래도 팬들한텐 잘했다
김지웅은 씨~발 논란 빼도…그냥 좀 그런게 많음
아무리 그래도 여자 면전에 대고 신발 소리 한 놈이랑 비교하냐 승한 최소한 팬들한텐 잘했어 능력치도 좋았고
미안한데 승한이는 쟤랑 비교당할정도 아니다
난 홍승한 개싫어했는데 김지웅 터지고 나니까 그건 아무것도 아니었다는 걸 깨닫음 최소한 걔는 팬들한테라도 잘했지,,
승한 빨면서 김가람 욕하는 년들 개웃김
후자는 아이돌로서 부주의한거지 일반인으로선 잘못한 건 하나도 없잖아.. 묶어서 그만 까자
당연히 승한 아니냐..
이런글왜쓰는거임 대체 재미도 없는데
빠혐하는 아이돌이 실존해도되냐? 활동중도아닌 데뷔전 일반인시절에 한 게 왜 지금까지 쌍욕먹는 건지도 ㅉㅉ
승앰ㅋㅋㅋ한명이 댓 쓰는 거 ㅈㄴ티난다
닥승한 팬들한텐 잘함
닥승한 얜 솔직히 지금 활중한것도 이해안감
승한
홍승한은 실력은 5새대 중 최상위권이라 그래도 나쁜건 아님
그래도 돈은 젤 많이 벌었을걸..
괜찮아

애초에 엔시티는 3세댄데 뭔소리야
라이즈 - 4개월 전에 데뷔하다
00 , 01 , 02 , 02 , 03 , 04
제베원 - 6개월 전에 데뷔하다
98 , 00 , 01 , 02 , 02 , 04 , 04 , 05 , 07
엔하이픈 - 3년 전에 데뷔하다
01 , 02 , 02 , 02 , 03 , 04 , 05


......엔하이픈은 아직 어리다
엔시티즌인데 엔시티 3세대임 ㅂㅁㄱ
한국에서 대중성 얻기 vs

In [ ]:
unique_data = list(set(comment_list))
len(unique_data)

948

In [ ]:
save_txt('nate_comment', unique_data)

데이터 저장 
